<a href="https://colab.research.google.com/github/gt-cse-6040/skills_oh_week_01/blob/main/week01_session01_NB02_function_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Asserting yourself: Program correctness and Function development ##

_Main topics covered during today's session:_

From the previous NB:
1. **Asymptotic running time (or cost)** and **cost (or work) efficiency**, e.g., algorithms that scale like $\mathcal{O}(n)$ vs. $\mathcal{O}(n^2)$
2. **Slicing notation**
3. **List comprehensions**
4. **Helper functions**

This NB:

5. **Assert yourself:** Does my code do what I think it should?



We did an impromptu demonstration in class inspired by the question, "How can we _prove_ that our program is correct?" The approach we took can sometimes be useful for debugging and even deriving efficient programs.

The main idea is to formalize any _facts_ that must be true at different parts in your program and then check whether the code makes those facts true or not.

For example, in the cumulative sums we know our goal is to take `x = [x[0], x[1], ..., x[n-1]]` and produce the output list `y = [y[0], y[1], ..., y[n-1]]`, where every `y[i] = x[0] + x[1] + ... + x[i]`. A natural starting point is to structure our solution like this:

In [ ]:
# Set a variable for the
# Running example, same as NB01:

x = [5, 3, -4, 20, 2, 9, 0, -1]

In [ ]:
def cumulative_sums__skel0(x):
    y = []
    for i in range(len(x)):
        # ...
        pass    # `pass` does nothing — it mollifies Python
        # ...
    return y

If our solution is correct, then we know that just before we `return y`, the condition embedded in the comment below must be true:

In [ ]:
def cumulative_sums__skel1(x):
    # Let `x == [x[0], x[1], ..., x[n-1]]`
    y = []
    for i in range(len(x)):
        pass
    # Fact: `len(y) == n and for all `0 <= i <= n-1`, `y[i] == x[0] + x[1] + ... + x[n-1]`
    return y

One nice thing about writing down this fact is that we already know one corner case—when `x` is empty—will be handled correctly. After all, `y` starts as empty; if `x` is also empty, so that `n == len(x) == 0`, the `for` loop will immediately complete leaving `y` empty but the conditions of the `Fact` satisfied naturally.

Next, focus on the loop skeleton in lines 4-5. What _must_ be true at the start of each iteration (right before `pass`) and at the end of each iteration (after `pass` but before the next iteration or loop-exit)?

In [ ]:
def cumulative_sums__skel2(x):
    # Let `x == [x[0], x[1], ..., x[n-1]]`
    y = []
    for i in range(len(x)): # `0 <= i <= n-1`
        # Fact A: `y[i-1] == x[0] + x[1] + ... + x[i-1]`
        pass
        # Fact B: `y[i]   == x[0] + x[1] + ... + x[i-1] + x[i]`
    # Fact C: `len(y) == n and for all `0 <= i <= n-1`, `y[i] == x[0] + x[1] + ... + x[n-1]`
    return y

Since `i` goes from `0` to `n-1`, `Fact A` and `Fact B` being true would imply `Fact C`.

Let's focus on just the `for` loop.
```python
    for i in range(len(x)):
        # Fact A: `y[i-1] == x[0] + x[1] + ... + x[i-1]`
        pass
        # Fact B: `y[i]   == x[0] + x[1] + ... + x[i-1] + x[i]`
```
What does `pass` need to look like in order for `Fact B` to follow from `Fact A`? Here's a good starting guess.

```python
    # Fact A: `y[i-1] == x[0] + x[1] + ... + x[i-1]`
    y[i] = y[i-1] + x[i]
    # Fact B: `y[i]   == x[0] + x[1] + ... + x[i-1] + x[i]`
```

While that makes sense, our code is not yet correct. The first problem is `y[i-1]`: the first time through the loop, `i=0` so `y[i-1]` does not exist!

Let's introduce a new variable, `s`, as a placeholder for the problematic `y[i-1]`:
```python
    y = []
    s = 0
    for i in range(len(x)):
        # Fact A: `s    == x[0] + x[1] + ... + x[i-1]`
        y[i] = s + x[i]
        # Fact B: `y[i] == x[0] + x[1] + ... + x[i-1] + x[i]`
```

Better! But `s` is not updated, so `Fact A` cannot remain true for `i > 0`. We need to update `s`, too.

```python
    y = []
    s = 0
    for i in range(len(x)):
        # Fact A: `s    == x[0] + x[1] + ... + x[i-1]`
        s += x[i]
        y[i] = s
        # Fact B: `y[i] == x[0] + x[1] + ... + x[i-1] + x[i]`
```

Closer! Now the problem is the update of `y[i]`: remember that `y` starts as empty, and you cannot assign an element of a list that does not yet exist.

Let's try two solutions: _preallocation_ of `y` and _appending_ to extend `y`.

In the first method, we know exactly how big the output needs to be, so we can _preallocate_ it. Then any reference to `y[i]` is safe as long as `i` is valid.
```python
    # Method 1: Preallocation
    y = [0] * len(x)  # `len(y) == `len(x)`
    s = 0
    for i in range(len(x)):
        # Fact A: `s    == x[0] + x[1] + ... + x[i-1]`
        s += x[i]
        y[i] = s
        # Fact B: `s    == x[0] + x[1] + ... + x[i-1] + x[i]`
        #          and `y[i] == s`
```
It's still easy to verify that `Fact B` follows from `Fact A` (and that `Fact C` from before follows, too).

In the second method, we append each new item as we go since such an operation exists for any Python list.
```python
    # Method 2: Appending
    y = []
    s = 0
    for i in range(len(x)):
        # Fact A: `s    == x[0] + x[1] + ... + x[i-1]`
        s += x[i]
        y.append(s)
        # Fact B: `s    == x[0] + x[1] + ... + x[i-1] + x[i]` and `y[i] == s`
```
It's a matter of taste which one you prefer.

To summarize, what we did above was to a) establish logical or mathematical facts that needed to hold at different points in the code, and then b) verify that the code does indeed imply those facts. This method is especially helpful for analyzing code that involves loops: what you are doing amounts to proof by induction, which you probably learned at some point in your distant mathematical-past.

There were two happy by-products, too!
1. Because we _started_ with the loop skeleton, writing down what we _expected_ let us _derive_ the code that we needed.
2. The code we derived is the _efficient_ (work-optimal) one that scales linearly, rather than quadratically, with the input size.
It's not easy to apply this technique and derive whole algorithms from scratch. But, it is an extremely useful technique when applied to small code fragments during debugging—it can help you isolate the code that violates your expectations.

## "There's a function for that..." ##

Of course, the standard Python library already has a function for calculating cumulative sums. It's called `accumulate` and is available via the `itertools` module:

In [ ]:
from itertools import accumulate

def cumulative_sum__v7(x):
    return list(accumulate(x))

cumulative_sum__v7(x)

> The `accumulate` function returns a _generator_. So to get the actual elements of the sequence, you need to "run" the generator. The implementation above does that by calling the `list` constructor.

### Beyond cumulative sums ###

The `accumulate` function can be used in more general settings than sums. For example, it can be used to do cumulative multiplies via the `func` parameter. The value of a `func` argument should be a function that can combine two elements. For example, a _cumulative product_, where the values are multiplied, would look like the following:

In [ ]:
print("* Sums:", *accumulate(x)) # cumulative sums

def multiply(a, b):
    return a * b

print("* Products:", *accumulate(x, func=multiply)) # cumulative products!

The `func` parameter, which is intended to refer to a _function_ (not a function's value when evaluated at a specific input), makes `accumulate` an example of a **higher-order function.** That is, it's a function that can accept another function or return a new function.

Why is that useful? It makes it possible to write one function that can be customized by the user through a simple interface. In this case, we can accumulate _anything_ as long we use a _binary operator_ to combine partial accumulated values with new values. By "binary operator," we really just mean some function `f(a,b)` that accepts two inputs and produces an output of the same type.

> The idea of higher-order functions and general operators can be viewed as _applied_ abstract algebra, which we would agree sounds a bit like an oxymoron.

## Summary ##

Here are the key ideas to review from today's notebooks.

1. **Achieving cost (or work) efficiency.** When designing a computational algorithm that operates on $n$ input objects and produces $k$ outputs, a good goal is _linear_ scaling. That means the computational algorithm takes $\mathcal{O}(n+k)$ steps. In this case, doubling the input or output will double the computational cost, which seems like a reasonable price to pay.

2. **Improving readability through basic idioms.** Python has many constructs that have led to common coding conventions, or _idioms_. Learning these idioms helps improve the readability of your code by making it more concise and using patterns that should be familiar to other Python programmers. The examples you saw here included **slicing for lists**, **higher-level functions** (like `sum` and `accumulate`), **list comprehensions** (use judiciously!), **helper functions**, and **zipper iteration**.

3. **Higher-order functions.** A powerful idea in software development, which has its roots in abstract algebra, is that of higher-order functions. These are functions that take other functions as input and use them. List comprehensions are one example: the comprehension `[f(e) for e in x]` works for any function `f` that can take the input `e`. The use of `accumulate` customized to do additions (the default), multiplications, minimums, or whatever it is you need to calculate the Fibonacci sequence, are all additional examples.